In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn

In [2]:
# Parameters
# ==================================================

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "1,2,3,4,5,6,8", "Comma-separated filter sizes (default: '1,2,3,4,5,6,8')")
tf.flags.DEFINE_string("num_filters", "50,100,150,150,200,200,200", "Number of filters per filter size (default: 50,100,150,150,200,200,200)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularizaion lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 32, "Batch Size (default: 32)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_string("checkpoint", '', "Resume checkpoint")
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=32
CHECKPOINT=
CHECKPOINT_EVERY=100
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=100
FILTER_SIZES=1,2,3,4,5,6,8
L2_REG_LAMBDA=0.0
LOG_DEVICE_PLACEMENT=False
NUM_CHECKPOINTS=5
NUM_EPOCHS=200
NUM_FILTERS=50,100,150,150,200,200,200



In [3]:
# Data Preparation
# ==================================================

In [4]:
# Load data
print("Loading data...")
x, y, vocabulary, vocabulary_inv = data_helpers.load_data()

Loading data...


In [5]:
x

array([[  9, 233,  15, ...,   0,   0,   0],
       [ 25,  13,   4, ...,   0,   0,   0],
       [ 89,  36,   4, ...,   0,   0,   0],
       ..., 
       [ 82, 480, 204, ...,   0,   0,   0],
       [110,  34,   2, ...,   0,   0,   0],
       [712,   7,   2, ...,   0,   0,   0]])

In [6]:
y

array([[0, 1],
       [0, 1],
       [0, 1],
       ..., 
       [1, 0],
       [1, 0],
       [1, 0]])

In [7]:
vocabulary

{u'\u5247': 3092,
 u'\u6000': 769,
 u'\u6302': 938,
 u'\ue785': 3983,
 u'\u8a87': 3792,
 u'\u3000': 1675,
 u'\u6d89': 1073,
 u'\u8c08': 1014,
 u'\u908b': 3448,
 u'\u4f0a': 2635,
 u'\u968f': 589,
 u'\u9810': 3016,
 u'\u8302': 2241,
 u'\u6297': 1643,
 u'\u8116': 3566,
 u'\u6599': 675,
 u'\u9785': 4238,
 u'\u7eca': 3567,
 u'\u4e9f': 3303,
 u'\u6d1e': 1259,
 u'\u9020': 745,
 u'\u54a3': 2930,
 u'\u9322': 2519,
 u'\u77a5': 3568,
 u'\u7624': 3984,
 u'\u822c': 227,
 u'\u86af': 2931,
 u'\u6f5c': 1760,
 u'\u6cb3': 1099,
 u'\u6b32': 1280,
 u'\u4fb5': 2520,
 u'\u4e34': 717,
 u'\u92b7': 3793,
 u'\u7136': 74,
 u'\u75b9': 3449,
 u'\u5438': 693,
 u'\u573a': 384,
 u'\u5bbd': 600,
 u'\u7ebf': 498,
 u'@': 2004,
 u'\u6342': 2932,
 u'\u9edb': 3725,
 u'\u53cd': 329,
 u'\u724c': 613,
 u'\u76cf': 2636,
 u'\u79d1': 915,
 u'\u9f39': 3569,
 u'\u7b52': 2187,
 u'\u568f': 3985,
 u'\u82d7': 2385,
 u'\u5eb8': 1644,
 u'\u82e5': 1143,
 u'\u8bdd': 146,
 u'\u984c': 2398,
 u'\u8d5e': 884,
 u'\u51e1': 1299,
 u'\u9060': 345

In [8]:
vocabulary_inv

['<PAD/>',
 u'\uff0c',
 u'\u7684',
 u'\u3002',
 u'\u662f',
 u'\u4e0d',
 u'\u4e86',
 u'\u4e00',
 u'\u6709',
 u'\u6211',
 u'\u5f88',
 u',',
 u'\u8fd9',
 u'\u4eba',
 u'\u597d',
 u'\u5728',
 u'\u4e2a',
 u'\u8fd8',
 u' ',
 u'.',
 u'\u5230',
 u'\u5c31',
 u'\u4e66',
 u'\u6ca1',
 u'\u4e5f',
 u'\u672c',
 u'\uff01',
 u'\u95f4',
 u'\u623f',
 u'\u770b',
 u'\u5e97',
 u'\u4e0a',
 u'\u6765',
 u'\u9152',
 u'\u5927',
 u'\u4ee5',
 u'\u65f6',
 u'\u8981',
 u'0',
 u'\u90fd',
 u'\u8bf4',
 u'\u53ef',
 u'\u70b9',
 u'\u80fd',
 u'\u5c0f',
 u'\u6bd4',
 u'\u591a',
 u'\u7528',
 u'1',
 u'\u5f97',
 u'\u670d',
 u'\u540e',
 u'\u8fc7',
 u'\u4eec',
 u'\u611f',
 u'\u4e48',
 u'\u4f4f',
 u'2',
 u'\u4e3a',
 u'\u4e0b',
 u'\u52a1',
 u'\u89c9',
 u'\u9519',
 u'\u4e2d',
 u'\u548c',
 u'\u5b50',
 u'\u4f46',
 u'\u5bf9',
 u'\u4f1a',
 u'\u53bb',
 u'\u4ef7',
 u'\u592a',
 u'\u81ea',
 u'\u3001',
 u'\u7136',
 u'\u4ed6',
 u'\u5929',
 u'\u51fa',
 u'\u9762',
 u'\u8f83',
 u'\u751f',
 u'\u91cc',
 u'\u7535',
 u'\u4e70',
 u'\u5b9e',
 u'\u5bb6',

In [9]:
# Randomly shuffle data
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = x[shuffle_indices]
y_shuffled = y[shuffle_indices]

In [10]:
# Split train/test set
# TODO: This is very crude, should use cross-validation
x_train, x_dev = x_shuffled[:-300], x_shuffled[-300:]
y_train, y_dev = y_shuffled[:-300], y_shuffled[-300:]
sequence_length = x_train.shape[1]
print("Vocabulary Size: {:d}".format(len(vocabulary)))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
print("Sequnence Length: {:d}".format(sequence_length))

Vocabulary Size: 4616
Train/Dev split: 34824/300
Sequnence Length: 2876


In [11]:
# Training
# ==================================================

with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=sequence_length,
            num_classes=2,
            vocab_size=len(vocabulary),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=map(int, FLAGS.filter_sizes.split(",")),
            num_filters=map(int, FLAGS.num_filters.split(",")),
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-4)
        grads_and_vars = optimizer.compute_gradients(cnn.loss, aggregation_method=2)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        #vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-1/W:0/grad/hist is illegal; using conv-maxpool-1/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-1/W:0/grad/sparsity is illegal; using conv-maxpool-1/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-1/b:0/grad/hist is illegal; using conv-maxpool-1/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-1/b:0/grad/sparsity is illegal; using conv-maxpool-1/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-2/W:0/grad/hist is illegal; using conv-maxpool-2/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-2/W:0/grad/sparsity is illegal; using conv-maxpool-2/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-2/b:0/grad/hist is illegal; using 

KeyboardInterrupt: 